# 🧠 OmniAGI - Google Colab Edition

**Framework AGI com GPU - Versão Estável**

---

## 1️⃣ Verificar GPU

In [1]:
import os, sys, torch

IN_COLAB = 'google.colab' in sys.modules
print(f"📍 Ambiente: {'Colab' if IN_COLAB else 'Local'}")

if torch.cuda.is_available():
    GPU_NAME = torch.cuda.get_device_name(0)
    VRAM_GB = torch.cuda.get_device_properties(0).total_memory / 1e9
    print(f"✅ GPU: {GPU_NAME}")
    print(f"✅ VRAM: {VRAM_GB:.1f} GB")
else:
    VRAM_GB = 0
    print("⚠️ Sem GPU")

📍 Ambiente: Colab
✅ GPU: Tesla T4
✅ VRAM: 15.8 GB


## 2️⃣ Instalar Dependências

In [2]:
if IN_COLAB:
    !pip install -q rwkv structlog chromadb sentence-transformers
    print("✅ Deps instaladas!")
else:
    print("📍 Local - deps já instaladas")

✅ Deps instaladas!


## 3️⃣ Clonar OmniAGI

In [3]:
import os, sys

if IN_COLAB:
    if not os.path.exists('/content/OmniAGI'):
        !git clone https://github.com/gabrielima7/OmniAGI.git /content/OmniAGI
    OMNIAGI_PATH = '/content/OmniAGI'
    os.chdir(OMNIAGI_PATH)
    !mkdir -p models/rwkv
else:
    OMNIAGI_PATH = '/media/zorin/HD/projetos/OmniAGI'
    os.chdir(OMNIAGI_PATH)

sys.path.insert(0, OMNIAGI_PATH)
print(f"📁 Path: {OMNIAGI_PATH}")

📁 Path: /content/OmniAGI


## 4️⃣ Configurar Modelo

⚠️ **IMPORTANTE**: Use modelo 1.6B para evitar crashes!

In [4]:
import os, torch

# SEMPRE usar 1.6B no Colab para estabilidade
# O modelo 3B causa crash no kernel
MODEL_SIZE = "1b6"
MODEL_PATH = "models/rwkv/rwkv-6-1b6.pth"
MODEL_URL = "https://huggingface.co/BlinkDL/rwkv-6-world/resolve/main/RWKV-x060-World-1B6-v2.1-20240328-ctx4096.pth"
STRATEGY = "cuda fp16"  # 1.6B cabe fácil na GPU

print(f"📦 Modelo: RWKV-6 {MODEL_SIZE.upper()}")
print(f"⚙️ Estratégia: {STRATEGY}")
print(f"📁 Existe: {os.path.exists(MODEL_PATH)}")

📦 Modelo: RWKV-6 1B6
⚙️ Estratégia: cuda fp16
📁 Existe: False


## 5️⃣ Download do Modelo

In [5]:
import os

if not os.path.exists(MODEL_PATH):
    print(f"⬇️ Baixando RWKV {MODEL_SIZE.upper()}...")
    os.makedirs(os.path.dirname(MODEL_PATH), exist_ok=True)
    !wget -q --show-progress -O {MODEL_PATH} {MODEL_URL}
    print("✅ Download concluído!")
else:
    print(f"✅ Modelo existe: {MODEL_PATH}")

print(f"📊 Tamanho: {os.path.getsize(MODEL_PATH) / 1e9:.2f} GB")

⬇️ Baixando RWKV 1B6...
models/rwkv/rwkv-6- 100%[===================>]   2.98G  26.2MB/s    in 2m 57s  
✅ Download concluído!
📊 Tamanho: 3.20 GB


## 6️⃣ Carregar Modelo

In [6]:
import torch
from rwkv.model import RWKV
from rwkv.utils import PIPELINE, PIPELINE_ARGS

# Limpar cache GPU antes de carregar
if torch.cuda.is_available():
    torch.cuda.empty_cache()

print(f"🔄 Carregando RWKV-6 {MODEL_SIZE.upper()}...")
model = RWKV(model=MODEL_PATH, strategy=STRATEGY)
pipeline = PIPELINE(model, 'rwkv_vocab_v20230424')
args = PIPELINE_ARGS(temperature=0.7, top_p=0.9)

print("✅ Modelo carregado!")
if torch.cuda.is_available():
    used = torch.cuda.memory_allocated() / 1e9
    print(f"📊 VRAM: {used:.1f} GB")

/usr/local/lib/python3.12/dist-packages/torch/backends/__init__.py:46: UserWarning: Please use the new API settings to control TF32 behavior, such as torch.backends.cudnn.conv.fp32_precision = 'tf32' or torch.backends.cuda.matmul.fp32_precision = 'ieee'. Old settings, e.g, torch.backends.cuda.matmul.allow_tf32 = True, torch.backends.cudnn.allow_tf32 = True, allowTF32CuDNN() and allowTF32CuBLAS() will be deprecated after Pytorch 2.9. Please see https://pytorch.org/docs/main/notes/cuda.html#tensorfloat-32-tf32-on-ampere-and-later-devices (Triggered internally at /pytorch/aten/src/ATen/Context.cpp:80.)
  self.setter(val)


🔄 Carregando RWKV-6 1B6...
RWKV_JIT_ON 1 RWKV_CUDA_ON 0 RESCALE_LAYER 6

Loading models/rwkv/rwkv-6-1b6.pth ...
Model detected: v6.0
Strategy: (total 24+1=25 layers)
* cuda [float16, float16], store 25 layers
0-cuda-float16-float16 1-cuda-float16-float16 2-cuda-float16-float16 3-cuda-float16-float16 4-cuda-float16-float16 5-cuda-float16-float16 6-cuda-float16-float16 7-cuda-float16-float16 8-cuda-float16-float16 9-cuda-float16-float16 10-cuda-float16-float16 11-cuda-float16-float16 12-cuda-float16-float16 13-cuda-float16-float16 14-cuda-float16-float16 15-cuda-float16-float16 16-cuda-float16-float16 17-cuda-float16-float16 18-cuda-float16-float16 19-cuda-float16-float16 20-cuda-float16-float16 21-cuda-float16-float16 22-cuda-float16-float16 23-cuda-float16-float16 24-cuda-float16-float16 
emb.weight                        f16      cpu  65536  2048       
blocks.0.ln1.weight               f16   cuda:0   2048             
blocks.0.ln1.bias                 f16   cuda:0   2048             

## 7️⃣ Testar Geração

In [7]:
import time

prompt = "Artificial General Intelligence is"
print(f"📝 Prompt: {prompt}")

start = time.time()
output = pipeline.generate(prompt, token_count=50, args=args)
elapsed = time.time() - start

print(f"\n🤖 Output: {output}")
print(f"\n⚡ {50/elapsed:.1f} tokens/s")

📝 Prompt: Artificial General Intelligence is

🤖 Output:  the foundational AI technology that will allow machines to learn from experience, reason and solve problems without being explicitly programmed. It is a blend of AI technologies such as machine learning, natural language processing, robotics, and other advances in computer science.


⚡ 12.5 tokens/s


## 8️⃣ Testar Módulos OmniAGI

In [8]:
print("🧪 TESTANDO MÓDULOS")
print("=" * 40)

# Consciousness
try:
    from omniagi.consciousness import ConsciousnessEngine
    c = ConsciousnessEngine()
    c.awaken()
    print(f"🧠 Consciousness: {c.reflect()['state']} ✅")
except Exception as e:
    print(f"🧠 Consciousness: {e} ❌")

# Math
try:
    from omniagi.benchmarks.arc_solver import ChainOfThoughtSolver
    s = ChainOfThoughtSolver()
    r = s.solve('sum', '25+37')
    print(f"📐 Math 25+37={r.answer} {'✅' if r.answer=='62' else '❌'}")
except Exception as e:
    print(f"📐 Math: {e} ❌")

# RAG
try:
    from omniagi.memory.rag import RAGSystem
    rag = RAGSystem('test')
    rag.initialize()
    print(f"📚 RAG: OK ✅")
except Exception as e:
    print(f"📚 RAG: {e} ❌")

print("=" * 40)

🧪 TESTANDO MÓDULOS
🧠 Consciousness: No module named 'llama_cpp' ❌
📐 Math: No module named 'llama_cpp' ❌
📚 RAG: No module named 'llama_cpp' ❌


## 9️⃣ Chat com AGI

In [9]:
def chat(prompt, tokens=80):
    p = f"User: {prompt}\nAssistant:"
    r = pipeline.generate(p, token_count=tokens, args=args)
    return r.split("User:")[0].strip() if "User:" in r else r.strip()

print("🤖 Chat")
for q in ["What is AGI?", "Calculate 15*3"]:
    print(f"\n👤 {q}")
    print(f"🤖 {chat(q)[:150]}")

🤖 Chat

👤 What is AGI?
🤖 AGI (Artificial General Intelligence) refers to an artificial intelligence system that can perform any intellectual task that a human can perform, wit

👤 Calculate 15*3
🤖 The result of 15 * 3 is 45.


## 🔟 Sua Pergunta

In [10]:
pergunta = "What is consciousness?"

print(f"👤 {pergunta}")
print(f"\n🤖 {chat(pergunta, tokens=100)}")

👤 What is consciousness?

🤖 Consciousness is the process of being aware or having awareness of something. It refers to the ability to perceive, think, and experience the world around us. Consciousness is a fundamental aspect of our existence as it allows us to make decisions, communicate, and interact with the environment around us.


---

**GitHub**: https://github.com/gabrielima7/OmniAGI

🌟 **OmniAGI** 🌟